In [15]:
import pandas as pd
import re

In [16]:
f = open('WhatsApp Chat with Subject.txt', 'r', encoding='utf-8')

In [17]:
data = f.read()

In [18]:
# print(data)

In [19]:
pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [20]:
messages = re.split(pattern, data)[2:]
# messages
# len(messages)

In [21]:
dates = re.findall(pattern, data)
# dates